# Comparative Linguistic Analysis of bioRxiv and PMC

This notebook is a copy of 02_corpora_comparison.ipynb where PLOS Biology reviewers requested that I rerun this analysis without special characters like ('%', '-', '\t', etc.).

In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import csv
import itertools
from pathlib import Path

import numpy as np
import pandas as pd
import pickle
import spacy
from scipy.stats import chi2_contingency
from tqdm import tqdm_notebook

from annorxiver_modules.corpora_comparison_helper import (
    aggregate_word_counts,
    dump_to_dataframe,
    get_term_statistics,
    KL_divergence,
)

# Full Text Comparison (Global)

## Gather Word Frequencies

In [2]:
biorxiv_count_path = Path("output/total_word_counts/biorxiv_total_count.tsv")
pmc_count_path = Path("output/total_word_counts/pmc_total_count.tsv")
nytac_count_path = Path("output/total_word_counts/nytac_total_count.tsv")

In [3]:
if not biorxiv_count_path.exists():
    biorxiv_corpus_count = aggregate_word_counts(
        list(Path("output/biorxiv_word_counts").rglob("*tsv"))
    )
    dump_to_dataframe(biorxiv_corpus_count, "output/biorxiv_total_count.tsv")
    biorxiv_corpus_count.most_common(10)

In [4]:
if not pmc_count_path.exists():
    pmc_corpus_count = aggregate_word_counts(
        list(Path("../../pmc/pmc_corpus/pmc_word_counts").rglob("*tsv"))
    )
    dump_to_dataframe(pmc_corpus_count, "output/pmc_total_count.tsv")
    pmc_corpus_count.most_common(10)

In [5]:
if not nytac_count_path.exists():
    nytac_corpus_count = aggregate_word_counts(
        list(Path("../../nytac/corpora_stats/output").rglob("*tsv"))
    )
    dump_to_dataframe(nytac_corpus_count, "output/nytac_total_count.tsv")
    nytac_corpus_count.most_common(10)

In [6]:
biorxiv_total_count_df = pd.read_csv(biorxiv_count_path.resolve(), sep="\t")

pmc_total_count_df = pd.read_csv(pmc_count_path.resolve(), sep="\t")

nytac_total_count_df = pd.read_csv(nytac_count_path.resolve(), sep="\t")

In [7]:
biorxiv_sentence_length = pickle.load(open("output/biorxiv_sentence_length.pkl", "rb"))
pmc_sentence_length = pickle.load(
    open("../../pmc/pmc_corpus/pmc_sentence_length.pkl", "rb")
)
nytac_sentence_length = pickle.load(
    open("../../nytac/corpora_stats/nytac_sentence_length.pkl", "rb")
)

In [8]:
spacy_nlp = spacy.load("en_core_web_sm")
stop_word_list = list(spacy_nlp.Defaults.stop_words)

## Get Corpora Comparison Stats

In [9]:
biorxiv_sentence_len_list = list(biorxiv_sentence_length.items())
biorxiv_data = {
    "document_count": len(biorxiv_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), biorxiv_sentence_len_list)),
    "token_count": biorxiv_total_count_df["count"].sum(),
    "stop_word_count": (
        biorxiv_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), biorxiv_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], biorxiv_sentence_len_list))))
    ),
    "negatives": (biorxiv_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        biorxiv_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        biorxiv_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / biorxiv_total_count_df["count"].sum(),
    "pronouns": (biorxiv_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (biorxiv_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / biorxiv_total_count_df["count"].sum(),
    "passives": (
        biorxiv_total_count_df.query(
            "dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']"
        )["count"].sum()
    ),
    "passive%": (
        biorxiv_total_count_df.query(
            "dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']"
        )["count"].sum()
    )
    / biorxiv_total_count_df["count"].sum(),
}

In [10]:
pmc_sentence_len_list = list(pmc_sentence_length.items())
pmc_data = {
    "document_count": len(pmc_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), pmc_sentence_len_list)),
    "token_count": pmc_total_count_df["count"].sum(),
    "stop_word_count": (
        pmc_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), pmc_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], pmc_sentence_len_list))))
    ),
    "negatives": (pmc_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        pmc_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        pmc_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / pmc_total_count_df["count"].sum(),
    "pronouns": (pmc_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (pmc_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / pmc_total_count_df["count"].sum(),
    "passives": (
        pmc_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    ),
    "passive%": (
        pmc_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    )
    / pmc_total_count_df["count"].sum(),
}

In [11]:
nytac_sentence_len_list = list(nytac_sentence_length.items())
nytac_data = {
    "document_count": len(nytac_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), nytac_sentence_len_list)),
    "token_count": nytac_total_count_df["count"].sum(),
    "stop_word_count": (
        nytac_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), nytac_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], nytac_sentence_len_list))))
    ),
    "negatives": (nytac_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        nytac_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        nytac_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / nytac_total_count_df["count"].sum(),
    "pronouns": (nytac_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (nytac_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / nytac_total_count_df["count"].sum(),
    "passives": (
        nytac_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    ),
    "passive%": (
        nytac_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    )
    / nytac_total_count_df["count"].sum(),
}

In [12]:
# This dataframe contains document statistics for each Corpus
# document count - the number of documents within the corpus
# Sentence count - the number of sentences within the corpus
# Token count - the number of tokens within the corpus
# Stop word counts - the number of stop words within the corpus
# Average document length - the average number of sentences within a document for a given corpus
# Average sentence length - the average number of words within a sentence for a given corpus
# Negatives - the number of negations (e.g. placing not in within a sentence) within a given corpus
# Coordinating Conjunctions - the number of coordinating conjunctions (and, but, for etc.) within a given corpus
# Pronouns - the number of pronouns within a given corpus
# Passive - the number of passive words within a given corpus

token_stats_df = pd.DataFrame.from_records(
    [biorxiv_data, pmc_data, nytac_data], index=["bioRxiv", "PMC", "NYTAC"]
).T
token_stats_df.to_csv("output/figures/corpora_token_stats.tsv", sep="\t")
token_stats_df

,bioRxiv,PMC,NYTAC
document_count,7.111800e+04,1.977647e+06,1.855658e+06
sentence_count,2.219574e+07,4.804898e+08,7.217104e+07
token_count,4.209699e+08,8.597101e+09,1.218673e+09
stop_word_count,1.584294e+08,3.153077e+09,5.593911e+08
avg_document_length,3.120973e+02,2.429604e+02,3.889242e+01
avg_sentence_length,2.270775e+01,2.146228e+01,1.989098e+01
negatives,1.148382e+06,2.492880e+07,7.272401e+06
coordinating_conjunctions,1.429574e+07,3.070823e+08,3.873005e+07
coordinating_conjunctions%,3.395904e-02,3.571929e-02,3.178050e-02
pronouns,4.604432e+06,7.499412e+07,4.671255e+07


## LogLikelihood + Odds Ratio + KL Divergence Calculations

The goal here is to compare word frequencies between bioRxiv and pubmed central. The problem when comparing word frequencies is that non-meaningful words (aka stopwords) such as the, of, and, be, etc., appear the most often. To account for this problem the first step here is to remove those words from analyses.

### Remove Stop words

In [13]:
biorxiv_total_count_df = (
    biorxiv_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
biorxiv_total_count_df

,lemma,count
2681961,the,22645292
2018157,of,14639457
656569,and,11981201
1572392,in,10132319
2704461,to,8146317
...,...,...
1542255,i.3sl,1
1542254,i.34,1
401503,478052_mir,1
401504,478090_mir,1


In [14]:
pmc_total_count_df = (
    pmc_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
pmc_total_count_df

,lemma,count
92606744,the,455469360
76187309,of,305683746
38780140,and,258669384
62245336,in,209051122
93273610,to,154688271
...,...,...
36210554,a2.7106faslfas,1
36210553,a2.710.47appropriateness3.500.710.500.00n,1
36210552,a2.710.014g,1
36210551,"a2.71.62.02.0f(3,139",1


In [15]:
nytac_total_count_df = (
    nytac_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
nytac_total_count_df

,lemma,count
1924366,the,72975471
1614479,of,33885977
781698,a,30376684
820301,and,29992626
1937587,to,29018148
...,...,...
1067648,creduto,1
1067650,creeblies,1
1067652,creechan,1
1067660,creedan,1


### Calculate LogLikelihoods and Odds ratios

In [17]:
biorxiv_vs_pmc = get_term_statistics(biorxiv_total_count_df, pmc_total_count_df, 100)

biorxiv_vs_pmc.to_csv(
    "output/comparison_stats/biorxiv_vs_pmc_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

biorxiv_vs_pmc

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,high,440917,9086405,420924101,8596698154,34.095825,0.991043
1,model,730123,8136668,420924101,8596698154,208130.617615,1.832641
2,genes,702833,7335919,420924101,8596698154,240898.860291,1.956706
3,observed,424926,7528396,420924101,8596698154,7786.766418,1.152760
4,values,347575,6047939,420924101,8596698154,8039.073547,1.173732
...,...,...,...,...,...,...,...
123,neurons,245673,1628424,420924101,8596698154,205354.667908,3.081185
124,min,144260,4947283,420924101,8596698154,44530.309570,0.595535
125,results,564580,13192663,420924101,8596698154,10217.549683,0.874019
126,state,212458,2461307,420924101,8596698154,53774.764282,1.762930


In [18]:
biorxiv_vs_nytac = get_term_statistics(
    biorxiv_total_count_df, nytac_total_count_df, 100
)
biorxiv_vs_nytac.to_csv(
    "output/comparison_stats/biorxiv_nytac_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)
biorxiv_vs_nytac

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,high,440917,710856,420924101,1218668568,8.797412e+04,1.795796
1,model,730123,89348,420924101,1218668568,1.473133e+06,23.658816
2,genes,702833,15385,420924101,1218668568,1.770927e+06,132.262419
3,observed,424926,26516,420924101,1218668568,9.692428e+05,46.396702
4,values,347575,53681,420924101,1218668568,6.610928e+05,18.746049
...,...,...,...,...,...,...,...
187,results,564580,150633,420924101,1218668568,8.879579e+05,10.851435
188,children,29066,633876,420924101,1218668568,2.164851e+05,0.132759
189,women,21344,502009,420924101,1218668568,1.775001e+05,0.123097
190,state,212458,1251630,420924101,1218668568,1.077170e+05,0.491450


In [19]:
pmc_vs_nytac = get_term_statistics(pmc_total_count_df, nytac_total_count_df, 100)

pmc_vs_nytac.to_csv(
    "output/comparison_stats/pmc_nytac_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

pmc_vs_nytac

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,high,9086405,710856,8596698154,1218668568,2.763675e+05,1.812026
1,model,8136668,89348,8596698154,1218668568,1.543475e+06,12.909686
2,observed,7528396,26516,8596698154,1218668568,1.753264e+06,40.248377
3,genes,7335919,15385,8596698154,1218668568,1.787933e+06,67.594438
4,values,6047939,53681,8596698154,1218668568,1.212017e+06,15.971317
...,...,...,...,...,...,...,...
188,results,13192663,150633,8596698154,1218668568,2.473892e+06,12.415559
189,children,2844519,633876,8596698154,1218668568,9.611810e+04,0.636148
190,women,3019814,502009,8596698154,1218668568,1.050943e+04,0.852752
191,state,2461307,1251630,8596698154,1218668568,1.128463e+06,0.278769


## Calculate KL Divergence

In [20]:
term_grid = [100, 200, 300, 400, 500, 1000, 1500, 2000, 3000, 5000]
kl_data = []
for num_terms in tqdm_notebook(term_grid):
    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                biorxiv_total_count_df, pmc_total_count_df, num_terms=num_terms
            ),
            "comparison": "biorxiv_vs_pmc",
        }
    )

    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                biorxiv_total_count_df, nytac_total_count_df, num_terms=num_terms
            ),
            "comparison": "biorxiv_vs_nytac",
        }
    )

    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                pmc_total_count_df, nytac_total_count_df, num_terms=num_terms
            ),
            "comparison": "pmc_vs_nytac",
        }
    )

In [21]:
kl_metrics = pd.DataFrame.from_records(kl_data)
kl_metrics.to_csv(
    "output/comparison_stats/corpora_kl_divergence_special_chars_removed.tsv",
    sep="\t",
    index=False,
)
kl_metrics

,num_terms,KL_divergence,comparison
0,100,0.022705,biorxiv_vs_pmc
1,100,0.209552,biorxiv_vs_nytac
2,100,0.819815,pmc_vs_nytac
3,200,0.030102,biorxiv_vs_pmc
4,200,1.156903,biorxiv_vs_nytac
5,200,1.110685,pmc_vs_nytac
6,300,0.036384,biorxiv_vs_pmc
7,300,1.484073,biorxiv_vs_nytac
8,300,1.298867,pmc_vs_nytac
9,400,0.042423,biorxiv_vs_pmc


# Preprint to Published View

In [22]:
mapped_doi_df = (
    pd.read_csv("../journal_tracker/output/mapped_published_doi.tsv", sep="\t")
    .query("published_doi.notnull()")
    .query("pmcid.notnull()")
    .groupby("preprint_doi")
    .agg(
        {
            "author_type": "first",
            "heading": "first",
            "category": "first",
            "document": "first",
            "preprint_doi": "last",
            "published_doi": "last",
            "pmcid": "last",
        }
    )
    .reset_index(drop=True)
)
mapped_doi_df.tail()

,author_type,heading,category,document,preprint_doi,published_doi,pmcid
30922,regular article,new results,microbiology,872325_v1.xml,10.1101/872325,10.1128/mbio.03197-19,PMC7078482
30923,regular article,new results,cell biology,872408_v1.xml,10.1101/872408,10.1186/s13072-020-00335-x,PMC7057672
30924,regular article,new results,ecology,872549_v1.xml,10.1101/872549,10.1093/aob/mcaa101,PMC7539359
30925,regular article,new results,biochemistry,872879_v1.xml,10.1101/872879,10.1038/s41467-020-14898-6,PMC7048817
30926,regular article,new results,developmental biology,873232_v1.xml,10.1101/873232,10.1534/g3.119.400967,PMC7056964


In [23]:
print(f"Total # of Preprints Mapped: {mapped_doi_df.shape[0]}")
print(f"Total % of Mapped: {mapped_doi_df.shape[0]/71118}")

Total # of Preprints Mapped: 30927
Total % of Mapped: 0.43486880958407154


In [24]:
preprint_count = aggregate_word_counts(
    [
        Path(f"output/biorxiv_word_counts/{Path(file)}.tsv")
        for file in mapped_doi_df.document.values.tolist()
        if Path(f"output/biorxiv_word_counts/{Path(file)}.tsv").exists()
    ]
)

preprint_count_df = (
    pd.DataFrame.from_records(
        [
            {
                "lemma": token[0],
                "pos_tag": token[1],
                "dep_tag": token[2],
                "count": preprint_count[token],
            }
            for token in preprint_count
        ]
    )
    .groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)

preprint_count_df.head()

,lemma,count
1475592,the,10336972
1109828,of,6697661
362762,and,5443114
863927,in,4638238
1487920,to,3757477


In [25]:
published_count = aggregate_word_counts(
    [
        Path(f"../../pmc/pmc_corpus/pmc_word_counts/{file}.tsv")
        for file in mapped_doi_df.pmcid.values.tolist()
        if Path(f"../../pmc/pmc_corpus/pmc_word_counts/{file}.tsv").exists()
    ]
)

published_count_df = (
    pd.DataFrame.from_records(
        [
            {
                "lemma": token[0],
                "pos_tag": token[1],
                "dep_tag": token[2],
                "count": published_count[token],
            }
            for token in published_count
        ]
    )
    .groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)

published_count_df.head()

,lemma,count
2113302,the,6818238
1654790,of,4378340
659905,and,3602889
1317069,in,2996432
2130676,to,2382655


In [26]:
preprint_vs_published = get_term_statistics(preprint_count_df, published_count_df, 100)

preprint_vs_published.to_csv(
    "output/comparison_stats/preprint_to_published_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

preprint_vs_published

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,high,200844,126000,193427969,130362488,399.301096,1.074290
1,genes,353237,239861,193427969,130362488,8.031273,0.992521
2,model,333119,215834,193427969,130362488,203.573996,1.040191
3,observed,198696,127579,193427969,130362488,182.850925,1.049647
4,values,157581,115514,193427969,130362488,468.183392,0.919396
...,...,...,...,...,...,...,...
100,based,225596,149164,193427969,130362488,32.806801,1.019297
101,neurons,118342,68822,193427969,130362488,957.395788,1.158897
102,min,65949,69779,193427969,130362488,6865.648548,0.636967
103,results,253927,165513,193427969,130362488,111.965830,1.033975


Main takeaways from this analysis:
1. On a global scale bioRxiv contains more field specific articles as top words consist of: neuron, gene, genome, network
2. "Patients" appear more correlated with PMC as most preprints involving patients are shipped over to medRxiv.
3. Many words associated with PMC are health related which ties back to the medRxiv note.
4. Citation styles change as preprints transition to published versions. Et Al. has a greater association within bioRxiv compared to PMC.
5. On a local scale published articles contain more statistical concepts (e.g., t-test) as well as quantitative measures (e.g. degree signs). (High associated lemmas are t, -, degree sign etc.)
6. Publish articles have a focus shift on mentioning figures, adding supplementary data etc compared to preprints.
7. Preprints have a universal way of citing published works by using the et al. citation. Hard to pinpoint if leading factor is because of peer review or journal style, but it will be an interesting point to discuss in the paper.